# Проверка непараметрических гипотез

In [ ]:
import warnings
warnings.filterwarnings("ignore")

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !wget -q -O requirements.txt https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/seminars/sem3/requirements.txt
    !pip install -q -r requirements.txt

## Библиотеки

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

import scipy.stats as st
from statsmodels.distributions.empirical_distribution import ECDF
from statsmodels.stats.descriptivestats import sign_test

import seaborn as sns
from permutation_test import permutationtest

# перестановочный критерий
from permute.core import one_sample, two_sample

# критерий знаков
from statsmodels.stats.descriptivestats import sign_test

# для перестановок
import itertools

# бутстрап
from arch.bootstrap import IIDBootstrap

## Мышки с зеркалами

#### Данные

In [ ]:
data_pd = pd.read_csv('https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/seminars/sem3/data/mirror_mice.txt', sep='\t', header=None)
data = data_pd.values.reshape(-1)

data_pd.head()

#### Гистограмма распределения

In [ ]:
plt.hist(data)
plt.show()

#### Биномиальный тест: ```st.binom_test```

In [ ]:
st.binom_test(np.sum(data>0.5), data.shape[0])

#### Критерий знаков

In [ ]:
sign_test(data, mu0=0.5)

#### Перестановочный критерий:
$H_0: \mathsf{E}X = m_0$

$H_1: \mathsf{E}X <\not=> m_0$

$T = \sum X_i-m_0$

In [ ]:
t = (data - 0.5)
T = np.sum(t)
T

In [ ]:
list(itertools.combinations(range(16), 2))

In [ ]:
list(itertools.product([-1, 1], repeat=2))

In [ ]:
count = 0
good = 0
for v in itertools.product([-1, 1], repeat=16):
    count +=1
    stat = np.dot(t, v)
    if abs(stat)>=abs(T):
        good+=1
    
        
good/count

##### Библиотека

In [ ]:
one_sample(data-0.5, alternative='two-sided')

In [ ]:
one_sample(data, 0.5*np.ones_like(data), alternative='two-sided')

#### Бутстрап

#### Вручную

In [ ]:
rs = np.random.RandomState(0)

idx = rs.randint(low=0, high=16, size=(1000, 16))
results = []
for i in idx:
    sample = data[i]
    results.append(sample.mean())

np.percentile(results, [2.5, 97.5])

#### Библиотека

In [ ]:
bs = IIDBootstrap(data)
bs.conf_int(np.median, 1000, method='basic')

## Терапия при анорексии
**Everitt N., McMahon M. (2004). Nutrition and the surgical patient. In Consensus in Clinical Nutrition, Cambridge: Cambridge University Press.**

В исследовании оценивается эффективность поведенческой терапии для лечения анорексии. Для 50 пациентов известен вес до начала терапии и по её окончании. Была ли терапия эффективной?

#### Данные

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/seminars/sem3/data/weight.txt', sep='\t')
w = data.values

data.head()

### Ввизуализация данных

In [ ]:
plt.scatter(w[:,0], w[:,1])
plt.xlabel('Before')
plt.ylabel('After')

plt.show()

### Проверка на нормальность

#### QQ-plot

In [ ]:
_ = st.probplot(w[:,1]-w[:,0], plot=plt)

#### Шапиро (```st.shapiro```)

In [ ]:
st.shapiro(w[:,1]-w[:,0])

#### Критерий знаков (```sign_test```)

In [ ]:
sign_test(w[:,1]-w[:,0])

#### Уилкоксон (```st.wilcoxon```):

In [ ]:
st.wilcoxon(w[:,1]-w[:,0])

#### Перестановочный критерий (```one_sample```):

In [ ]:
one_sample(w[:,1]-w[:,0],  alternative='two-sided')

#### Доверительный интервал для среднего прироста веса с помощью бутстрепа:

In [ ]:
bs = IIDBootstrap(w[:,1]- w[:,0])
print(bs.conf_int(np.mean, 1000, method='bca'))

#### Доверительный интервал для медианы прироста веса:

In [ ]:
bs = IIDBootstrap(w[:,1]- w[:,0])
print(bs.conf_int(np.median, 1000, method='bca'))

## Недвижимость в Сиэттле
**Hesterberg T., Monaghan S., Moore D.S., Clipson A., Epstein R. (2005). Bootstrap methods and permutation tests. In Introduction to the Practice of Statistics, New York: W. H. Freeman and Company.** http://statweb.stanford.edu/~tibs/stat315a/Supplements/bootstrap.pdf

Имеются данные о продажной стоимости недвижимости в Сиэтле для 50 сделок в 2001 году и 50 в 2002. Изменились ли цены?

#### Данные

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/seminars/sem3/data/seattle.txt', delimiter='\t')
data.head()

#### Boxplot

In [ ]:
sns.boxplot(data=data, y="Price", x="Year")

In [ ]:
x1 = data.loc[data['Year']==2001]['Price'].values
x2 = data.loc[data['Year']==2002]['Price'].values

In [ ]:
sns.kdeplot(x1, label='2001')
sns.kdeplot(x2, label='2002')
plt.legend(loc="best")
plt.show()

#### QQ-график

In [ ]:
_ = st.probplot(x1, plot=plt)

In [ ]:
_ = st.probplot(x2, plot=plt)

#### Критерий Стьюдента:

In [ ]:
st.ttest_ind(x1, x2)

#### Критерий Манна-Уитни-Уилкоксона:

In [ ]:
st.mannwhitneyu(x1, x2, alternative= 'two-sided')

#### Перестановочный критерий:

In [ ]:
two_sample(x1, x2,  alternative='two-sided')

#### Доверительные интервалы для средних значений в выборках с помощью бутстрепа:

Ожидаемые ответы ```[246, 330], [241,403]```

In [ ]:
bs = IIDBootstrap(x1)
print(bs.conf_int(np.mean, 1000, method='bca'))

bs = IIDBootstrap(x2)
print(bs.conf_int(np.mean, 1000, method='bca'))

#### Доверительный интервал для разности двух средних с помощью бутстрепа:

Ожидаемые ответы: ```[-52, 115]```

In [ ]:
bs = IIDBootstrap(x2-x1)
print(bs.conf_int(np.mean, 1000, method='bca'))

#### Статистика Колмогорова-Смирнова

In [ ]:
st.ks_2samp(x1, x2)

## Алюминий в тополях
**Laureysens I., Blust R., De Temmerman L., Lemmens C., Ceulemans R. (2004). Clonal variation in heavy metal accumulation and biomass production in a poplar coppice culture. I. Seasonal variation in leaf, wood and bark concentrations. Environmental Pollution, 131, 485-494.**

Для 13 разновидностей тополей, растущих в зоне интенсивного загрязнения, в
августе и ноябре измерялась средняя концентрация алюминия в микрограммах на
грамм древесины.

In [ ]:
al = pd.read_csv('https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/seminars/sem3/data/poplar.txt', delimiter='\t')
al.head()

In [ ]:
plt.scatter(al['November'].values, al['August'].values)
plt.xlabel('November')
plt.ylabel('August')
plt.show()

In [ ]:
delta = al['August']-al['November']

#### QQ график

In [ ]:
_ = st.probplot(delta, plot=plt)

#### Критерий знаков

In [ ]:
sign_test(delta)

#### Критерий знаковых рангов Уилкоксона

In [ ]:
st.wilcoxon(delta)

#### Перестановочный критерий

In [ ]:
one_sample(al['November']-al['August'],  alternative='two-sided')

#### Поменяем статистику (вместо ```mean``` поставим ```t```)

Ожидаемый ответ: ```0.03```

In [ ]:
# Ваш код

#### Доверительный интервал для средней разности с помощью бутстрепа

In [ ]:
bs = IIDBootstrap(delta)
bs.conf_int(np.mean, 1000, method='basic')

#### Доверительный интервал методом 'bca'

In [ ]:
bs = IIDBootstrap(delta)
bs.conf_int(np.mean, 1000, method='bca')

#### Зависимость устойчивости доверительного интервала от количества попыток сэмплирования (проверить для n<20000)

In [ ]:
for id, i in enumerate([5,10,50,100,500,1000, 5000, 10000]):
    bs = IIDBootstrap(delta)
    ci = bs.conf_int(np.mean, i, method='basic')
    plt.plot([np.log(i), np.log(i)], ci )

In [ ]:
for id, i in enumerate([5,10,50,100,500,1000, 5000, 10000, 20000]):
    bs = IIDBootstrap(delta)
    ci = bs.conf_int(np.mean, i, method='bca')
    plt.plot([np.log(i), np.log(i)], ci)
    print(ci)

#### Зависимость доверительного интервала от уровня доверия

In [ ]:
for p in [0.1, 0.2, 0.3,0.5, 0.7, 0.8, 0.9, 0.95, 0.99]:
    bs = IIDBootstrap(delta)
    ci = bs.conf_int(np.mean, 5000,size=p, method='bca')
    plt.plot([p, p], ci )
    print (ci)

## Время ремонта телекоммуникаций

Verizon is the primary local telephone company (the legal term is Incumbent Local Exchange Carrier, ILEC) for a large area in the eastern United States. As such, it is responsible for providing repair service for the customers of other telephone companies (known as Competing Local Exchange Carriers, CLECs) in this region. Verizon is subject to fines if the repair times (the time it takes to fix a problem) for CLEC customers are substantially worse than those for Verizon’s own customers. This is determined using hypothesis tests, negotiated with the local Public Utilities Commission (PUC).

#### Данные

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/seminars/sem3/data/verizon.txt', delimiter='\t')
data.head()

#### Boxplot

In [ ]:
sns.boxplot(data=data,x='Time', y='Group')

In [ ]:
x1 = data.loc[data['Group']=='ILEC']['Time'].values
x2 = data.loc[data['Group']=='CLEC']['Time'].values
x1.shape, x2.shape

#### Гистограмма

In [ ]:
plt.hist(x1, bins=10, density=True, label="ILEC")
plt.hist(x2, bins=10, density=True, label='CLEC')
plt.legend(loc="best")
plt.show()

#### QQ-plot

In [ ]:
st.probplot(x1, plot=plt)
plt.title('ILEC')
plt.show()

In [ ]:
st.probplot(x2, plot=plt)
plt.title('CLEC')
plt.show()

#### Критерий Мана-Уитни-Уилкоксона

In [ ]:
st.mannwhitneyu(x1, x2, alternative='two-sided')

#### Перестановочный критерий

In [ ]:
two_sample(x1, x2, reps=1000, alternative='two-sided')

#### Критерий Колмогора-Смирнова

In [ ]:
st.ks_2samp(x1,x2)